<a href="https://colab.research.google.com/github/Diligent0924/ToxicClassification/blob/master/%EC%B5%9C%EC%A2%85_%EB%8F%85%EC%84%B1%EC%97%AC%EB%B6%80%ED%99%95%EC%9D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U huggingface_hub==0.0.8
!pip install transformers==4.6.0
!pip install datasets==1.11.0
!pip install gc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 29.0 MB/s 
     |████████████████████████████████| 3.3 MB 49.1 MB/s 
     |████████████████████████████████| 880 kB 54.3 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=9a230e8531711d9a8ba1c0a807e95b1098ff8a80446a0882538a13ca07281935
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 264 kB 30.6 MB/s 
     |████████████████████████████████| 115 kB 66.8 MB/s 
     |████████████████████████████████| 212 kB 72.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-py

In [ ]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
final_data = pd.read_csv('Toxic_Sevtoxic_4000.csv')

In [ ]:
final_data

,Unnamed: 0,id,comment_text,toxic,severe_toxic,classification
0,36587,61c614d37720c9f4,be a toilet. so everyone can piss her off. hey...,1,0,0
1,191204,7e193f28aa406c52,If you fidn this page then \n == GO TO THELL ...,1,0,0
2,68945,b878709393b479c8,"Fuck you, I was curious as to who the actors p...",1,0,0
3,60785,a2b50de88da494ef,"""\n\nHey man, I was just playing. No worries, ...",1,0,0
4,42348,70fbe89b516ae59c,Hey dude if you want to suck cock please dont ...,1,0,0
...,...,...,...,...,...,...
3884,222931,fd6868579ae07fc2,"== SLUT == \n\n I hope you get breast cancer, ...",1,1,1
3885,223030,fdbc343a337959d1,:: No one wants children seeing an explicit ph...,1,1,1
3886,223059,fde3334312a8ea67,== ur dumb == \n\n you jerk you blocked me! i ...,1,1,1
3887,223209,fe9315bad284186b,"""b style=""""font-size:40px"""">CALVIN IS A FUCKIN...",1,1,1


In [ ]:
# Sentence 내에서 특수문자를 제거하는 곳 => 나중에 PLM까지 건드리게 된다는 형태소 분석까지 행해야할 수도 있음
import re

def clean_text(inputString):
  text_rmv = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!·|]', ' ', inputString) 
  text_rmv = ' '.join(text_rmv.split())
  return text_rmv

feature = [clean_text(str(s)) for s in final_data['comment_text']]

In [ ]:
import pandas as pd
import datasets
from datasets import Dataset, DatasetDict

df = pd.DataFrame({'text' : feature, 'label' : final_data['classification']})
df = Dataset.from_pandas(df)

In [ ]:
len(final_data['classification'])

3889

In [ ]:
# Data Set을 분할
from sklearn.model_selection import train_test_split

train_feature, test_feature, train_target, test_target = train_test_split(feature, final_data['classification'], train_size = 0.7, random_state=77)

train_feature, val_feature, train_target, val_target = train_test_split(train_feature, train_target, train_size = 0.7, random_state=77) # train_size = 0.5, val_size = 0.2 , test_size = 0.3

# 데이터 분할을 통하여 train_set, val_set, test_set으로 나눈다.

In [ ]:
import pandas as pd
import datasets
from datasets import Dataset, DatasetDict

train = pd.DataFrame({'text' : train_feature, 'label' : train_target})
validation = pd.DataFrame({'text' : val_feature, 'label' : val_target})
test = pd.DataFrame({'text' : test_feature, 'label' : test_target})

train = Dataset.from_pandas(train)
validation = Dataset.from_pandas(validation)
test = Dataset.from_pandas(test)
df = DatasetDict()

df['train'] = train
df['validation'] = validation
df['test'] = test

In [ ]:
df

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 1905
    })
    validation: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 817
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 1167
    })
})

In [ ]:
# Sentence를 Word로 변환시켜 분할하는 방법 => 보통 Word 분할 시에는 매우 많은 데이터가 필요로 하므로 bert-base-uncased와 같이 Google에서 이미 만든 PLM을 사용함
from transformers import BertTokenizerFast, TFBertModel

BERT_MODEL = 'bert-base-uncased'
MAX_LEN = 128

tokenizer = BertTokenizerFast.from_pretrained(BERT_MODEL)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding = True)

tokenized_set = df.map(preprocess_function, batched = True)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
tokenized_set

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'attention_mask', 'input_ids', 'label', 'text', 'token_type_ids'],
        num_rows: 1905
    })
    validation: Dataset({
        features: ['__index_level_0__', 'attention_mask', 'input_ids', 'label', 'text', 'token_type_ids'],
        num_rows: 817
    })
    test: Dataset({
        features: ['__index_level_0__', 'attention_mask', 'input_ids', 'label', 'text', 'token_type_ids'],
        num_rows: 1167
    })
})

In [ ]:
!pip install wandb
import wandb
wandb.login()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 28.5 MB/s 
     |████████████████████████████████| 181 kB 74.5 MB/s 
     |████████████████████████████████| 158 kB 61.7 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 157 kB 59.9 MB/s 
     |████████████████████████████████| 157 kB 74.0 MB/s 
     |████████████████████████████████| 157 kB 69.1 MB/s 
     |████████████████████████████████| 157 kB 75.9 MB/s 
     |████████████████████████████████| 157 kB 74.6 MB/s 
     |████████████████████████████████| 157 kB 69.5 MB/s 
     |████████████████████████████████| 157 kB 80.0 MB/s 
     |████████████████████████████████| 156 kB 76.4 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=1005a42557f755afef4c2d982db50f0bedec38cf83e57f32207eb7c117e9a05d
  Stored in directory: /root/.cache/pip/wheels/3e/31

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Model을 Training하는 방법에는 2가지가 있는데 1번째는 Model을 일정한 lr에 따라서 학습하는 것이고 두번쨰는 lr의 속도를 조절하면서 학습하는 것이다.
# lr의 속도를 조절하는 방법을 scheduler라고 하며 이러한 방법으로 Warm_up(얼마만큼 학습률을 조절할것인가)를 판단할 수 있다.
# 이번 데이터셋에서는 scheduler를 이용한 데이터 학습을 진행할 예정이다.

# W&B dashBoard를 이용하여 데이터를 확인하기
# BERT의 기본 Model에 layer / Node / drop_out 등을 조정할 수 있게 하는 Model

from transformers import BertConfig, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np

from datasets import load_metric 

training_args = TrainingArguments( # Default => learning_rate = 5e^5, optimizer = AdamW, loss = model.compute_loss
    output_dir='./BERT_BASE',          # tensorboard에 ./result형태로 loging들이 모여있음
    num_train_epochs= 5,              # 얼마나 epoch를 할 것인가
    per_device_train_batch_size= 8,  # 32정도
    per_device_eval_batch_size= 8,   # 
    warmup_steps= 5000,                # scheduler를 몇단계로 학습시킬 것인가 (최적화 방법을 찾아봐야한다.)
    weight_decay= 0.01,               # warm_up시에 얼마 강도로 줄 것인가 (Layer에 대한 이야기라고 생각되는데 확실한 것은 찾아봐야 알 수 있을듯...)
    logging_dir='./logs',            # log 장소를 의미하는데 해당 log를 어디서 확인할 수 있는지가 안나와있음 => log안에 어떤 log가 들어있는지 파악이 안됨
    logging_steps = 10,
    evaluation_strategy= 'epoch'
)


def get_model():
  config = BertConfig.from_pretrained(BERT_MODEL, 
                                        hidden_dropout_prob = 0.2,
                                        attention_probs_dropout_prob = 0.2,
                                        num_lables= 2)
  model = AutoModelForSequenceClassification.from_pretrained(BERT_MODEL, config=config)
  return model

model = get_model()

def compute_metrics(eval_pred):
    metric = load_metric('glue','mrpc')
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)



trainer = Trainer(                     # TFtrainer는 model, args(
    model= model,                 # 어떤 model을 사용하여 학습할 것인지를 확인하는 방법
    args=training_args,                  # training arguments, defined above
    train_dataset= tokenized_set['train'],         # 학습할 때만 사용되는 데이터셋을 의미하는듯
    eval_dataset= tokenized_set['validation'],             # eval_dataset이 TFtrainer에서는 어떤 역할을 맡고 있는가?
    compute_metrics = compute_metrics    # 어떤 방법으로 정확도를 나타낼 것인가를 확인하는 방법
)

trainer.train()

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.660500,0.644685,0.565483,0.710204
2,0.558300,0.501267,0.756426,0.803165
3,0.356100,0.429417,0.805386,0.833158
4,0.409100,0.405228,0.822521,0.841876
5,0.426700,0.458556,0.810282,0.836670


Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

TrainOutput(global_step=1195, training_loss=0.4921916504784109, metrics={'train_runtime': 1199.8645, 'train_samples_per_second': 0.996, 'total_flos': 522326959822848.0, 'epoch': 5.0, 'init_mem_cpu_alloc_delta': 1601720320, 'init_mem_gpu_alloc_delta': 439072256, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 413323264, 'train_mem_gpu_alloc_delta': 1375686144, 'train_mem_cpu_peaked_delta': 2056192, 'train_mem_gpu_peaked_delta': 5391650816})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
MODEL_NAME = 'My-bert-base'
MODEL_SAVE_PATH = os.path.join("_model", MODEL_NAME) # change this to your preferred location

if os.path.exists(MODEL_SAVE_PATH):
    print(f"{MODEL_SAVE_PATH} -- Folder already exists \n")
else:
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    print(f"{MODEL_SAVE_PATH} -- Folder create complete \n")

# save tokenizer, model
model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

from transformers import TextClassificationPipeline

# Load Fine-tuning model
loaded_tokenizer = BertTokenizerFast.from_pretrained(MODEL_SAVE_PATH)
loaded_model = TFBertForSequenceClassification.from_pretrained(MODEL_SAVE_PATH)

text_classifier = TextClassificationPipeline(
    tokenizer=loaded_tokenizer, 
    model=loaded_model, 
    framework='tf',
    return_all_scores=True,
    device= 0
)

_model/My-bert-base -- Folder create complete 



NameError: ignored

대회 제출 데이터 불러오기

In [ ]:
import pandas as pd

comments_to_score = pd.read_csv(path[0]+'/comments_to_score.csv',encoding='latin-1') 

In [ ]:
df = comments_to_score
# Sentence 내에서 특수문자를 제거하는 곳 => 나중에 PLM까지 건드리게 된다는 형태소 분석까지 행해야할 수도 있음
import re

def clean_text(inputString):
  text_rmv = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!·|]', ' ', inputString) 
  text_rmv = ' '.join(text_rmv.split())
  return text_rmv

df['text'] = df['text'].apply(lambda x:clean_text(x))

In [ ]:
#for row in df.itertuples():

In [ ]:
def predict_text(text):
    preds_list = text_classifier(text)[0]
    sorted_preds_list = sorted(preds_list, key=lambda x: x['score'], reverse=True)
    label = sorted_preds_list[0]['label'] # label
    score = sorted_preds_list[0]['score'] # score

    return [preds_list,label , score]


In [ ]:
# 예시
result = predict_text('be a toilet. so everyone can piss her off.')
result

In [ ]:
from tqdm import tqdm
tqdm.pandas()
df['predict'] = df['text'].progress_apply(lambda x:predict_text(x))

In [ ]:
# 이후작업 --> 확률값 기준으로 정렬( 라벨 1  확률 내림차순/ 라벨 0 내림차순)

In [ ]:
df['label'] = df['predict'].progress_apply(lambda x:x[1])
df['proba'] = df['predict'].progress_apply(lambda x:x[2])

In [ ]:
df